# 형태소 분석기 테스트 - Pykomoran
Konlpy가 굉장히 메모리 리소스를 많이 차지하는 것으로 추정. 배포에서도 이때문에 문제가 발생하는 것으로 보아 메모리를 줄이기 위해 Pykomoran을 이용해보고자 함. 성능이 괜찮을 경우 변경 예정.
## 참고링크
https://pydocs.komoran.kr/firststep/installation.html#id7

## 설치

In [1]:
!pip install PyKomoran

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.8/198.8 kB 16.3 MB/s eta 0:00:00
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [2]:
from PyKomoran import *
komoran = Komoran("STABLE")

In [3]:
komoran.get_plain_text("여기서는 이렇다고 말했어요. 그리고 여기서는 증가했습니다. 누구를 가르킵니다")

'여기/NP 서/JKB 는/JX 이렇/VA 다고/EC 말/NNG 하/XSV 았/EP 어요/EF ./SF 그리고/MAJ 여기/NP 서/JKB 는/JX 증가/NNG 하/XSV 았/EP 습니다/EF ./SF 누구/NP 를/JKO 가르키/VV ㅂ니다/EC'

아래와 같이 초급 표현이 포함되어 있는지 확인을 해준다. 반복된 횟수만큼 감점한다.
근거: ‘-습니다., -요’로 문장 끝맺으면 감점된다. 이것은 초급 표현이기 때문이다

In [4]:
komoran.get_plain_text("이 정보는 이렇다고 합니다. 이 도표에서는 34%로 말했습니다. 이 정보를 괜찮아요.")

'이/MM 정보/NNG 는/JX 이렇/VA 다고/EC 하/VX ㅂ니다/EF ./SF 이/MM 도표/NNG 에서/JKB 는/JX 34/SN %/SW 로/JKB 말/NNG 하/XSV 았/EP 습니다/EF ./SF 이/MM 정보/NNG 를/JKO 괜찮/VA 아요/EF ./SF'

In [5]:
result = (komoran.get_plain_text("이 정보는 이렇다고 합니다. 이 도표에서는 34%로 말했습니다. 이 정보를 괜찮아요.")).split(' ')
result
for i in range(len(result)):
  temp = result[i].split('/')
  result[i] = temp
print(result)

[['이', 'MM'], ['정보', 'NNG'], ['는', 'JX'], ['이렇', 'VA'], ['다고', 'EC'], ['하', 'VX'], ['ㅂ니다', 'EF'], ['.', 'SF'], ['이', 'MM'], ['도표', 'NNG'], ['에서', 'JKB'], ['는', 'JX'], ['34', 'SN'], ['%', 'SW'], ['로', 'JKB'], ['말', 'NNG'], ['하', 'XSV'], ['았', 'EP'], ['습니다', 'EF'], ['.', 'SF'], ['이', 'MM'], ['정보', 'NNG'], ['를', 'JKO'], ['괜찮', 'VA'], ['아요', 'EF'], ['.', 'SF']]


In [6]:
result = (komoran.get_plain_text("신입회원을 모집합니다.")).split(' ')
result
for i in range(len(result)):
  temp = result[i].split('/')
  result[i] = temp
print(result)
result = (komoran.get_plain_text("신입회원을 모입니다")).split(' ')
result
for i in range(len(result)):
  temp = result[i].split('/')
  result[i] = temp
print(result)

[['신입', 'NNG'], ['회원', 'NNG'], ['을', 'JKO'], ['모집', 'NNG'], ['하', 'XSV'], ['ㅂ니다', 'EF'], ['.', 'SF']]
[['신입', 'NNG'], ['회원', 'NNG'], ['을', 'JKO'], ['모이', 'VV'], ['ㅂ니다', 'EC']]


# TFIDF에서의 PyKomoran 적용하기

In [7]:
!pip install -U scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 85.3 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer


vectorizer = TfidfVectorizer(min_df = 1, decode_error = 'ignore')

def sentence_token(contents):
  contents_tokens = list()
  for i in range(len(contents)):
    result = []
    test = (komoran.get_plain_text(contents[i])).split(' ')
    for j in range(len(test)):
      temp = test[j].split('/')
      test[j] = temp[0]
    #print('여기',test)
    contents_tokens.append(test)
  #print('최종', contents_tokens)


  contents_for_vectorize = []
  for content in contents_tokens:
    sentence = ''
    for word in content:
      sentence = sentence + ' ' + word
    contents_for_vectorize.append(sentence)

  #tf-idf 벡터화
  X = vectorizer.fit_transform(contents_for_vectorize)
  num_samples, num_features = X.shape
  #print(X)
  return X

In [9]:
testing = ['작년 대비 소비자 물가가 33% 증가했다.']
sentence_token(testing)

<1x6 sparse matrix of type '<class 'numpy.float64'>'
	with 6 stored elements in Compressed Sparse Row format>

In [10]:
def new_token(new_post):
  #new_post_tokens = [t.morphs(row) for row in new_post]
  test = (komoran.get_plain_text(new_post[0])).split(' ')
  new_post_tokens = []
  for j in range(len(test)):
      temp = test[j].split('/')
      new_post_tokens.append(temp[0])
  #print(new_post_tokens)

  new_post_for_vectorize = []
  sentence = ' '.join(new_post_tokens)
  new_post_for_vectorize.append(sentence)

  new_post_vec = vectorizer.transform(new_post_for_vectorize)
  #print('여기', new_post_vec)
  return new_post_vec

In [11]:
testing = ['작년 대비 소비자 물가가 33% 증가했다.']
new_token(testing)

<1x6 sparse matrix of type '<class 'numpy.float64'>'
	with 6 stored elements in Compressed Sparse Row format>

In [12]:
import scipy as sp
def dist_raw(v1,v2):
  delta = v1 - v2
  return sp.linalg.norm(delta.toarray())

In [13]:
def check_distance(X, new_post_vec, contents):
  best_doc = None
  best_dist = 65535
  best_i = None
  result = []
  for i in range(len(contents)):
    post_vec = X.getrow(i)
    d = dist_raw(post_vec, new_post_vec)
    #print(d)
    print('== Post %i with dist = %.2f : %s' %(i,d, contents[i]))
    if d < best_dist:
      best_dist = d
      best_i = i
      result = []
    if d == best_dist:
      result.append(contents[i])
  return best_i, best_dist, result

In [14]:
contents = [
    "작년 대비 소비자물가가 33% 증가했다.",
    "작년 대비 소비자물가가 33%증가했다.",
    "작년 대비 소비자물가가 15% 증가했다.",
    "내 나이는 올해 33살이다.",
    "문방구가서 펜 33개만 사와줄래?",
    "소비자물가가 크게 오를 전망이다.",
    "작년에 비해 소비자물가가 증가했다.",
    "작년에 비해 소비자물가가 33% 증가했다."
]
new_post = ["작년 대비 소비자물가가 33% 증가했다."]

X = sentence_token(contents)
new_post_vec = new_token(new_post)
#print('비교', X, new_post_vec)
best_i, best_dist, result = check_distance(X, new_post_vec, contents)
print("Best post is %i, dist=%.2f" %(best_i, best_dist))
print('-->', new_post)
print('---->', contents[best_i])

== Post 0 with dist = 0.00 : 작년 대비 소비자물가가 33% 증가했다.
== Post 1 with dist = 0.00 : 작년 대비 소비자물가가 33%증가했다.
== Post 2 with dist = 0.74 : 작년 대비 소비자물가가 15% 증가했다.
== Post 3 with dist = 1.27 : 내 나이는 올해 33살이다.
== Post 4 with dist = 1.31 : 문방구가서 펜 33개만 사와줄래?
== Post 5 with dist = 1.24 : 소비자물가가 크게 오를 전망이다.
== Post 6 with dist = 0.90 : 작년에 비해 소비자물가가 증가했다.
== Post 7 with dist = 0.77 : 작년에 비해 소비자물가가 33% 증가했다.
Best post is 0, dist=0.00
--> ['작년 대비 소비자물가가 33% 증가했다.']
----> 작년 대비 소비자물가가 33% 증가했다.
